In [1]:
# -*- coding: utf-8 -*-
"""
Created on Mon Jan 12 13:58:41 2026

@author: ksearle
"""

import pandas as pd
import xpress as xp
import os
import zipfile


zip_path = "CaseStudyDataPY.zip"
data_dir = "CaseStudyDataPY"

In [2]:
# -----------------------------------------------------------------------------
# Create directory and extract CSVs if it doesn't exist
# -----------------------------------------------------------------------------
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall(data_dir)

In [3]:
# -----------------------------------------------------------------------------
# Read supplier data
# The first column is used as the supplier index
# -----------------------------------------------------------------------------
Suppliers_df = pd.read_csv(f"{data_dir}/Suppliers.csv", index_col=0)
# Maximum supplier index (assumed to be integer-indexed)
nbSuppliers = Suppliers_df.index.max()

In [4]:
# -----------------------------------------------------------------------------
# Read postcode district data (used to define customers)
# -----------------------------------------------------------------------------
PostcodeDistricts = pd.read_csv(f"{data_dir}/PostcodeDistricts.csv", index_col=0)
# -----------------------------------------------------------------------------

# Read candidate facility data
# -----------------------------------------------------------------------------
Candidates_df = pd.read_csv(f"{data_dir}/Candidates.csv", index_col=0)
# Maximum candidate index
nbCandidates = Candidates_df.index.max()

In [5]:
# -----------------------------------------------------------------------------
# Read distance matrices
# Supplier → District distances
# District → District distances
# Column names are converted from strings to integers for correct .loc indexing
# -----------------------------------------------------------------------------
DistanceSupplierDistrict_df = pd.read_csv(
    f"{data_dir}/Distance Supplier-District.csv", index_col=0
)
DistanceSupplierDistrict_df.columns = DistanceSupplierDistrict_df.columns.astype(int)

DistanceDistrictDistrict_df = pd.read_csv(
    f"{data_dir}/Distance District-District.csv", index_col=0
)
DistanceDistrictDistrict_df.columns = DistanceDistrictDistrict_df.columns.astype(int)

In [6]:
# -----------------------------------------------------------------------------
# Read aggregate demand data (no time dimension)
# Creates a dictionary keyed by (Customer, Product)
# -----------------------------------------------------------------------------
Demand_df = pd.read_csv(f"{data_dir}/Demand.csv")
Demand = (
    Demand_df
        .set_index(["Customer", "Product"])["Demand"]
        .to_dict()
)

In [7]:
# -----------------------------------------------------------------------------
# Read demand data with time periods
# Creates a dictionary keyed by (Customer, Product, Period)
# -----------------------------------------------------------------------------
DemandPeriods_df = pd.read_csv(f"{data_dir}/DemandPeriods.csv")
DemandPeriods = (
    DemandPeriods_df
        .set_index(["Customer", "Product", "Period"])["Demand"]
        .to_dict()
)

# Number of time periods
nbPeriods = DemandPeriods_df["Period"].max()

In [8]:
# -----------------------------------------------------------------------------
# Read demand data with time periods and scenarios
# Creates a dictionary keyed by (Customer, Product, Period, Scenario)
# -----------------------------------------------------------------------------
DemandPeriodsScenarios_df = pd.read_csv(f"{data_dir}/DemandPeriodScenarios.csv")
DemandPeriodsScenarios = (
    DemandPeriodsScenarios_df
        .set_index(["Customer", "Product", "Period", "Scenario"])["Demand"]
        .to_dict()
)

# Number of scenarios
nbScenarios = DemandPeriodsScenarios_df["Scenario"].max()

In [9]:
# =============================================================================
# Index sets
# =============================================================================
Customers  = PostcodeDistricts.index
Candidates = Candidates_df.index
Suppliers  = Suppliers_df.index


# =============================================================================
# Vehicle-related data
# Vehicles are indexed as:
#   1 = 18t trucks
#   2 = 7.5t lorries
#   3 = 3.5t vans
# =============================================================================

# Vehicle capacity in tonnes
VehicleCapacity = {
    1: 9.0,
    2: 2.4,
    3: 1.5
}

# Cost in pounds per mile travelled (fixed cost)
VehicleCostPerMileOverall = {
    1: 1.666,
    2: 1.727,
    3: 1.285
}

# Cost in pounds per mile and tonne transported (variable cost)
VehicleCostPerMileAndTonneOverall = {
    1: 0.185,
    2: 0.720,
    3: 0.857
}

# CO₂ emissions in kg per mile and tonne transported
VehicleCO2PerMileAndTonne = {
    1: 0.11,
    2: 0.31,
    3: 0.30
}


In [10]:
# -----------------------------------------------------------------------------
# Time periods and scenarios
# -----------------------------------------------------------------------------
Times = range(1, nbPeriods + 1)
Scenarios = range(1, nbScenarios + 1)


# =============================================================================
# Transport cost calculations
# =============================================================================

# Cost from suppliers to candidate facilities
# Round-trip distance (factor 2)
# Cost depends on supplier vehicle type
# Division by 1000 converts from kg to tonnes
CostSupplierCandidate = {
    (k, j): 2
    * DistanceSupplierDistrict_df.loc[k, j]
    * VehicleCostPerMileAndTonneOverall[
        Suppliers_df.loc[k, "Vehicle type"]
    ]
    / 1000
    for j in Candidates
    for k in Suppliers
}

# Cost from candidate facilities to customers
# All transports use 3.5t vans (vehicle type 3)
# CostCandidateCustomers = {
#     (j, i): 2
#     * DistanceDistrictDistrict_df.loc[j, i]
#     * VehicleCostPerMileAndTonneOverall[3]
#     / 1000
#     for j in Candidates
#     for i in Customers
# }
# Cost from candidate facilities to clusters (already aggregated)



In [11]:
# =============================================================================
# Candidate parameters (from Candidates_df)
# =============================================================================

Setup = {
    j: Candidates_df.loc[j, "Setup cost"]
    for j in Candidates
}

Operating = {
    j: Candidates_df.loc[j, "Operating"]
    for j in Candidates
}

CapacityCandidates = {
    j: Candidates_df.loc[j, "Capacity"]
    for j in Candidates
}

CapacitySuppliers = {
    s: Suppliers_df.loc[s, "Capacity"]
    for s in Suppliers
}

ProductSuppliers= {
    s: Suppliers_df.loc[s, "Product group"]
    for s in Suppliers
}

In [12]:
DemandAgg_df = pd.read_csv(f"{data_dir}/DemandAggScenarios_20.csv")    
# Dictionary keyed by (Cluster, Product, Period)
DemandAgg = (
    DemandAgg_df
        .set_index(["Cluster", "Product", "Period", "Scenario"])["Demand"]
        .to_dict()
)

C_agg_long_df = pd.read_csv(f"{data_dir}/C_agg_long.csv")

C_agg_long_df["Warehouse"] = C_agg_long_df["Warehouse"].astype(int)
C_agg_long_df["Cluster"] = C_agg_long_df["Cluster"].astype(int)
C_agg_long_df["Cost"] = C_agg_long_df["Cost"].astype(float)

CostCandidateClusters = (
    C_agg_long_df
    .set_index(["Warehouse","Cluster"])["Cost"]
    .to_dict()
)
Clusters = sorted(DemandAgg_df["Cluster"].unique())
Products = sorted(DemandAgg_df["Product"].unique())

DemandAgg_df["Cluster"] = DemandAgg_df["Cluster"].astype(int)
DemandAgg_df["Product"] = DemandAgg_df["Product"].astype(int)
DemandAgg_df["Period"] = DemandAgg_df["Period"].astype(int)


# (j, i) = (candidate, cluster)

In [13]:
DemandAgg_df = DemandAgg_df[DemandAgg_df["Scenario"] <= 10]
# DemandAgg_df

In [14]:
# =====================
# SETS
# =====================
J = sorted(C_agg_long_df["Warehouse"].unique())
# J= J[:50]
I = sorted(DemandAgg_df["Cluster"].unique())
S = list(Suppliers_df.index)    # Suppliers
P = Products     # Products
T =  Times   # Time periods
Scenarios = sorted(DemandAgg_df["Scenario"].unique())


In [16]:
# =====================
# PARAMETERS
# =====================
# Demand: DemandPeriods[i,p,t]
# Warehouse costs: Setup[j], Operating[j], CapacityCandidates[j]
# Supplier capacity: CapacitySuppliers[s]
# Product supplied by supplier: ProductSuppliers[s]
# Transport costs: CostCandidateCustomers[j,i], CostSupplierCandidate[s,j]

# (Assumed to be defined as dictionaries)

In [17]:
# =============================================================================
# Build optimization model
# =============================================================================


prob = xp.problem("Assignment 1-b")

# =====================
# DECISION VARIABLES
# =====================

# Warehouse decisions
prob = xp.problem()


y = {(j,t): prob.addVariable(vartype=xp.binary) for j in J for t in T}
w = {(j,t): prob.addVariable(vartype=xp.binary) for j in J for t in T}
x = {(i,j,t,sc): prob.addVariable(vartype=xp.binary)
     for i in I for j in J for t in T for sc in Scenarios}
z = {(s,j,p,t,sc): prob.addVariable(vartype=xp.continuous, lb=0)
     for s in S for j in J for p in P for t in T for sc in Scenarios}





/var/folders/tn/v19h6ld131n5p0bkj6b850sr0000gn/T/ipykernel_2473/2095535132.py:6: LicenseWarning: Using the license file found in your Xpress installation. If you want to use this license and no longer want to see this message, use the following code before using the xpress module:
  xpress.init('/Applications/FICO Xpress/xpressmp/bin/xpauth.xpr')
  prob = xp.problem("Assignment 1-b")


In [18]:
# =====================
# OBJECTIVE FUNCTION
# =====================
Q = (
    xp.Sum(Setup[j] * y[j,t] for j in J for t in T) +
    xp.Sum(Operating[j] * w[j,t] for j in J for t in T) +
    (1/len(Scenarios)) *
    xp.Sum(
        CostCandidateClusters[j,i] * DemandAgg[i,p,t,sc] * x[i,j,t,sc]
        for i in I for j in J for p in P for t in T for sc in Scenarios
    ) +
    (1/len(Scenarios)) *
    xp.Sum(
        CostSupplierCandidate[s,j] * z[s,j,p,t,sc]
        for s in S for j in J for p in P for t in T for sc in Scenarios
    )
)
prob.setObjective(Q, sense=xp.minimize)


In [ ]:

# =====================
# CONSTRAINTS
# =====================


# Each customer served by exactly one warehouse per period
for i in I:
    for t in T:
        for sc in Scenarios:
            prob.addConstraint(
                xp.Sum(x[i,j,t,sc] for j in J) == 1
            )


# Assignment only to operating warehouses
for i in I:
    for j in J:
        for t in T:
            for sc in Scenarios:
                prob.addConstraint(x[i,j,t,sc] <= w[j,t])



# Build at most once
for j in J:
    prob.addConstraint(xp.Sum(y[j,t] for t in T) <= 1)


# Operating equals cumulative build
for j in J:
    for t in T:
        prob.addConstraint(w[j,t] == xp.Sum(y[j,s] for s in T if s <= t))

        
# Warehouse capacity
for j in J:
    for t in T:
        for sc in Scenarios:
            prob.addConstraint(
                xp.Sum(DemandAgg[i,p,t,sc] * x[i,j,t,sc] for i in I for p in P)
                <= CapacityCandidates[j] * w[j,t]
            )


# Flow balance at warehouses (by product)
for j in J:
    for p in P:
        for t in T:
            for sc in Scenarios:
                prob.addConstraint(
                    xp.Sum(z[s,j,p,t,sc] for s in S if ProductSuppliers[s] == p)
                    == xp.Sum(DemandAgg[i,p,t,sc] * x[i,j,t,sc] for i in I)
                )

# Supplier capacity per period 
for s in S:
    for t in T:
        for sc in Scenarios:
            prob.addConstraint(
                xp.Sum(z[s,j,p,t,sc] for j in J for p in P)
                <= CapacitySuppliers[s]
            )

In [20]:

prob.setObjective(Q, sense=xp.minimize)
# To turn on and off the solver log
xp.setOutputEnabled(True)


xp.setOutputEnabled(True)
prob.solve()


FICO Xpress v9.7.0, Hyper, solve started 21:16:25, Feb 15, 2026
Heap usage: 5008MB (peak 5008MB, 89KB system)
Minimizing MILP noname using up to 8 threads and up to 8192MB memory, with these control settings:
OUTPUTLOG = 1
NLPPOSTSOLVE = 1
XSLP_DELETIONCONTROL = 0
XSLP_OBJSENSE = 1
Original problem has:
   3537640 rows     12636800 cols     38137000 elements   3308800 entities
Presolved problem has:
   3508961 rows      5612561 cols     30908963 elements   3280561 entities
LP relaxation tightened
Presolve finished in 116 seconds
Heap usage: 10GB (peak 14GB, 89KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  8.00e+06] / [ 3.42e-03,  2.64e+02]
  RHS and bounds [min,max] : [ 1.00e+00,  1.53e+07] / [ 1.00e+00,  7.49e+03]
  Objective      [min,max] : [ 2.45e-05,  6.25e+06] / [ 6.27e-03,  6.88e+06]
Autoscaling applied Curtis-Reid scaling

Will try to keep branch and bound tree memory usage below 5.5GB
Starting 

SolverError: ?9999 Error: Critical error encountered during the MIP solve. Please check your log for the cause.

In [ ]:
#This happen because VPN was disconnected

In [21]:
# =============================================================================
# Post-processing and data visualisation
# =============================================================================

sol_status = prob.attributes.solstatus

if sol_status == xp.SolStatus.OPTIMAL:
    print("Optimal solution found")
    best_obj = prob.attributes.objval
    best_bound = prob.attributes.bestbound
    mip_gap = abs(best_obj - best_bound) / (1e-10 +abs(best_obj))
    print(f"MIP Gap: {mip_gap*100:.2f}%")
    
elif sol_status == xp.SolStatus.FEASIBLE:
    print("Feasible solution (not proven optimal)")
    best_obj = prob.attributes.objval
    best_bound = prob.attributes.bestbound
    mip_gap = abs(best_obj - best_bound) / (1e-10 +abs(best_obj))
    print(f"MIP Gap: {mip_gap*100:.2f}%")
elif sol_status == xp.SolStatus.INFEASIBLE:
    print("Model is infeasible")
elif sol_status == xp.SolStatus.UNBOUNDED:
    print("Model is unbounded")
else:
    print("No solution available")

Feasible solution (not proven optimal)
MIP Gap: 0.73%


In [ ]:
# # Write full model (objective + constraints) to LP file
# prob.write("model.lp", "lp")

# # Or to MPS format  
# prob.write("model.mps", "mps")


In [22]:
Q_optimal = prob.getSolution(Q)
Q_optimal

33578252.91696307

In [23]:
print("\n=== WAREHOUSES BUILT ===")
built = []
for j in J:
    for t in T:
        if prob.getSolution(y[j,t]) > 0.5:  # binary, so >0.5 means 1
            built.append((j, t))
            print(f"Warehouse {j} built at period {t}")

print(f"\nTotal warehouses ever built: {len(built)}")



=== WAREHOUSES BUILT ===
Warehouse 19 built at period 7
Warehouse 42 built at period 1
Warehouse 47 built at period 8
Warehouse 48 built at period 4
Warehouse 54 built at period 6
Warehouse 183 built at period 3
Warehouse 315 built at period 1
Warehouse 339 built at period 7
Warehouse 351 built at period 9
Warehouse 411 built at period 3

Total warehouses ever built: 10


In [ ]:
#----------------------------------
# COST BREAKDOWN
#----------------------------------

# --- 1. Construction Cost ---
construction_cost = sum(
    Setup[j] * prob.getSolution(y[j,t])
    for j in J for t in T
)

# --- 2. Operating Cost ---
operating_cost = sum(
    Operating[j] * prob.getSolution(w[j,t])
    for j in J for t in T
)


In [ ]:
# ---------------------------------
# PRINT RESULTS
# ----------------------------------
print("\n==============================")
print("COST BREAKDOWN")
print("==============================")
print(f"Construction Cost : £{construction_cost:,.2f}")
print(f"Operating Cost    : £{operating_cost:,.2f}")
total_cost = Q_optimal - (construction_cost + operating_cost)
print(f"Transport Cost    : £{total_cost:,.2f}")



COST BREAKDOWN
Construction Cost : £8,082,000.00
Operating Cost    : £5,311,800.00
Transport Cost    : £20,184,452.92


In [30]:
# # AFTER prob.optimize()
# print("=== WAREHOUSE OPENING SCHEDULE ===")
# print(f"{'Warehouse':<10} {'T=1':<6} {'T=2':<6} {'T=3':<6} ...")  # adjust for your |T|

# # Count open warehouses per period
# open_per_period = {t: 0 for t in T}
# warehouse_schedule = {}

# for j in J:
#     row = f"W{j:<7}"
#     status = []
#     for t in T:
#         w_val = prob.getSolution(w[j,t])
#         open_per_period[t] += w_val  # 1.0 if open, 0.0 if closed
#         status.append(f"{w_val:>5.0f}")
#         row += f"  {w_val:>4.0f}"
#     warehouse_schedule[j] = status
#     print(row)

# print("\n=== SUMMARY ===")
# print("Open warehouses per period:", open_per_period)
# print(f"Total warehouse-periods open: {sum(open_per_period.values())}")


In [29]:


# # Warehouse opening matrix
# w_solution = pd.DataFrame(
#     [[prob.getSolution(w[j,t]) for t in T] for j in J],
#     index=[f"W{j}" for j in J],
#     columns=[f"T{t}" for t in T]
# )

# print("Warehouse Opening Matrix (1=open, 0=closed):")
# print(w_solution.round(2))

# # Count open per period
# print("\nOpen warehouses per period:")
# print(w_solution.sum(axis=0).astype(int))


In [ ]:
# print("\n=== TOP CUSTOMER ASSIGNMENTS (sample) ===")
# for i in list(I)[:5]:  # first 5 customers
#     for t in T[:3]:     # first 3 periods
#         assigned = [(j, prob.getSolution(x[i,j,t])) 
#                    for j in J if prob.getSolution(x[i,j,t]) > 0.5]
#         if assigned:
#             print(f"Customer {i}, T={t}: {assigned}")
